In [42]:
#import dependencies
import pandas as pd
import matplotlib as plt
import scipy.stats as st

In [43]:
#Read in FEMA data
fema = pd.read_csv('publicAssistanceData/byDisasterandCounty.csv')
fema.head()

,incidentType,state,county,incidentCount,FEMA$
0,Chemical,WestVirginia,Boone,6,12013.226667
1,Chemical,WestVirginia,Cabell,2,3826.870000
2,Chemical,WestVirginia,Kanawha,12,50986.351667
3,Chemical,WestVirginia,Lincoln,9,7432.647778
4,Chemical,WestVirginia,Putnam,13,5519.388462


In [44]:
states = {"Alabama":'AL',"Alaska":'AK',"Arizona":'AZ',"Arkansas":'AR',"California":'CA',"Colorado":"CO",
  "Connecticut":"CT","Delaware":"DE","Florida":"FL","Georgia":"GA","Hawaii":"HI","Idaho":"ID","Illinois":"IL",
  "Indiana":"IN","Iowa":"IA","Kansas":"KS","Kentucky":"KY","Louisiana":"LA","Maine":"ME","Maryland":"MD",
  "Massachusetts":"MA","Michigan":"MI","Minnesota":"MN","Mississippi":"MS","Missouri":"MI","Montana":"MT",
  "Nebraska":"NE","Nevada":"NV","NewHampshire":"NH","NewJersey":"NJ","NewMexico":"NM","NewYork":"NY",
  "NorthCarolina":"NC","NorthDakota":"ND","Ohio":"OH","Oklahoma":"OK","Oregon":"OR","Pennsylvania":"PA",
  "RhodeIsland":"RI","SouthCarolina":"SC","SouthDakota":"SD","Tennessee":"TN","Texas":"TX","Utah":"UT",
  "Vermont":"VT","Virginia":"VA","Washington":"WA","WestVirginia":"WV","Wisconsin":"WI","Wyoming":"WY", "DistrictofColumbia":"DC"}
df_states = fema['state']
state_fix = [states[state] for state in df_states]
fema['state'] = state_fix
fema = fema.set_index(['state', 'county'])
fema.head()

incidentType  incidentCount         FEMA$
state county                                           
WV    Boone       Chemical              6  12013.226667
      Cabell      Chemical              2   3826.870000
      Kanawha     Chemical             12  50986.351667
      Lincoln     Chemical              9   7432.647778
      Putnam      Chemical             13   5519.388462

In [46]:
#Read in SAPIE data
sapie = pd.read_csv('SAPIE_DataClean/ByYear/18SAPIE.csv')
#Clean SAPIE Data
sapie['Name'] = sapie['Name'].str.strip('County')
sapie = sapie.drop(['Unnamed: 0', 'State County FIPS'], axis=1)
sapie = sapie.rename(columns={'Postal Code':'state', 'Name':'county'})
sapie.head()

,state,county,Median Household Income
0,AL,Autauga,44241
1,AL,Baldwin,41961
2,AL,Barbour,26062
3,AL,Bibb,32924
4,AL,Blount,37758
